In [ ]:
from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

import numpy as np

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})
import matplotlib.pyplot as plt

from jpasLAEs.utils import flux_to_mag

In [ ]:
cat = fits.open('UV_slopes_COLA1_field_2fil.fits')[1].data
cat.columns

In [ ]:
def continuum_powerlaw(wavelength, norm, beta, redshift):
    return norm*(wavelength/(1500.*(1+redshift)))**beta

def app_m_to_flux_nu(m):
    """
    :param m: apparent magnitude
    :return: the (observed) flux density
    """
    return 10 ** (-(m + 48.60) / 2.5)


def flux_to_lum(flux, z):
    """
    :param flux: flux
    :param z: redshift
    :return: luminosity
    """
    return flux * 4 * np.pi * cosmo.luminosity_distance(z).to("cm").value ** 2

def flux_nu_to_lum_nu(flux_nu, z):
    """
    :param flux_nu: the (observed) flux density
    :param z: redshift
    :return: the (intrinsic) luminosity density
    """
    return flux_to_lum(flux_nu, z) / (1 + z)

In [ ]:
N_iter = 1000

N_sources = len(cat)

M_UV = np.empty(N_sources)
M_UV_err_up = np.empty(N_sources)
M_UV_err_down = np.empty(N_sources)

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat['beta'][src], cat['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm'][src], cat['norm_err'][src], size=N_iter)

    this_redshift = cat['z_O3doublet_combined_n'][src]

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift) * 0.47 / 0.4

    this_M_UV_percentiles = np.percentile(this_M_UV,[16, 50, 84])

    M_UV[src] = this_M_UV_percentiles[1]
    M_UV_err_up[src] = this_M_UV_percentiles[2] - M_UV[src]
    M_UV_err_down[src] = M_UV[src] - this_M_UV_percentiles[0]

In [ ]:
src = cat['NUMBER'] == 9269

mask = ((np.asarray(cat['F115W_AUTO_fnu'] / cat['F115W_AUTO_enu']) > 5)
        & (np.asarray(cat['F150W_AUTO_fnu'] / cat['F150W_AUTO_enu']) > 5)
        & (cat['z_O3doublet'] > 0))

fig, ax = plt.subplots(figsize=(6, 4))

ax.errorbar(M_UV[mask], cat['beta'][mask],
            yerr=cat['beta_err'][mask],
            xerr=[M_UV_err_down[mask], M_UV_err_up[mask]],
            ls='', fmt='o', label='C1F [OIII] emitters',
            ecolor='lightsteelblue', mfc='dimgray', mec='dimgray')
ax.errorbar(M_UV[src], cat['beta'][src],
            yerr=cat['beta_err'][src],
            zorder=999, fmt='D', ms=10, capsize=5, mfc='r', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='COLA1')

xx = np.arange(-23, -18.8, 0.01)
xx_2 = np.arange(-18.8, -16, 0.01)
yy = -2.22 + -0.24*(xx - -18.8)
yy_2 = -2.22 + -0.08*(xx_2 - -18.8)

ax.plot(xx, yy, color='indigo', ls='--', lw=3,
        label=r'LBGs $z\sim6$ (Bouwens+14)')
ax.plot(xx_2, yy_2, color='indigo', ls='--', lw=3)

ax.set_ylim(-5, 1)
ax.set_xlim(-22, -18)

ax.set_xlabel(r'$M_{\rm UV}$')
ax.set_ylabel(r'$\beta_{\rm UV}$')

ax.legend(fontsize=11, loc='upper right')

ax.tick_params(direction='in', which='both')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.invert_xaxis()

savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
fig.savefig(f'{savefig_path}/beta_powerlaw.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')

plt.show()

In [ ]:
print(M_UV[src], M_UV_err_down[src], M_UV_err_up[src])